# Download Sentinel-1 data
- This notebook finds which eo-patches we need based on a chosen region defined with polygons from a geojson file 

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import pyproj
import gzip
import shutil
import glob
import os
from eolearn.core import (
    EOTask,
    EOPatch,
    EONode,
    EOWorkflow,
    OverwritePermission,
    LoadTask,
    SaveTask,
    FeatureType,
    linearly_connect_tasks,
    CreateEOPatchTask
)
from sentinelhub import CRS, BBoxSplitter
from eolearn.io import SentinelHubInputTask
from sentinelhub import DataCollection, SHConfig, ResamplingType
from sentinelhub.data_collections import _Resolution

import datetime
from tqdm import tqdm

from util_paths import SWEDEN
from util_paths import HARVEST_17_20_AOI_FILE
from util_paths import SENTINEL_1_11x11_PATH
from util_paths import UTM33N

# Use Area Of Interest (AOI) to find which eo-patches we need to download
 - AOI file should be a geojson with multipolygon features

In [ ]:
aoi = gpd.read_file(HARVEST_17_20_AOI_FILE)

In [ ]:
harvest_aoi = aoi
aoi_shape = harvest_aoi.geometry.values[-1]

## Also load multipolygon of Sweden to view the selection

In [ ]:
sweden_gdf = gpd.read_file(SWEDEN)
sweden_gdf = sweden_gdf.to_crs(crs = UTM33N)

In [ ]:
def aoi_grid(aoi, target_crs=UTM33N, grid_size=2500, save=False):
    """
    read in shape file and reproject it into the projection that will compute correct aoi size
    
    Args:
        shp: the AOI shapfile either in ESRI shapefile or geojson
        t_crs: the target coordination;
        gr_sz: tile/grid size to split the AOI, default is 168 by 168 pixels;
        save: save the generated AOI tile grid as a pickle file
    Return:
        patchID: the splitted tile that will be saved as EOpatch with the IDs

    Note:
        when save is set to Ture. An ESRI shapefile is saved to the disk under folder called "aoi_tile_grid"
    """
    #reading shapefile using geopandas
    aoi_geo = gpd.read_file(aoi)
    #reproject the AOI to a target CRS that will give a correct AOI size in m2
    aoi_reprj = aoi_geo.to_crs(crs=target_crs)
    #get the AOI geometry
    aoi_shape = aoi_reprj.geometry.values[-1]
    # get the width and height (sentinel-2 in 10m resolution)
    s2_res = 10
    width_pix = int((aoi_shape.bounds[2] - aoi_shape.bounds[0])/s2_res)
    heigth_pix = int((aoi_shape.bounds[3] - aoi_shape.bounds[1])/s2_res)
    print('Dimension of the area is {} x {} pixels'.format(width_pix, heigth_pix))
    width_grid = int(round(width_pix/grid_size))
    heigth_grid = int(round(heigth_pix/grid_size))

    # split the tile grid by the desired grid number
    tile_splitter = BBoxSplitter([aoi_shape], target_crs, (width_grid, heigth_grid))
    print("The area is splitted into a grid with {} by {} tiles!".format(width_grid, heigth_grid))

    tiles = np.array(tile_splitter.get_bbox_list())
    info_list = np.array(tile_splitter.get_info_list())

    # get the all polygon inform  i AOI
    idxs_x = [info['index_x'] for info in tile_splitter.info_list]
    idxs_y = [info['index_y'] for info in tile_splitter.info_list]

    #save all the patch ID for tiles and save it as numpy array
    patchID = np.array(range(len(tiles))).astype("int")
    geometry = [Polygon(bbox_.get_polygon()) for bbox_ in tiles[patchID]]

    return patchID, tiles, tile_splitter

In [ ]:
patch_ids, tile_list, bbox_splitter = aoi_grid(SWEDEN)

In [ ]:
bbox_list = np.array(bbox_splitter.get_bbox_list())
info_list = np.array(bbox_splitter.get_info_list())

# Prepare info of selected EOPatches
geometry = [Polygon(bbox.get_polygon()) for bbox in bbox_list]
idxs = [list(info_list).index(info) for info in info_list]
#idxs = [info.index for info in info_list]
idxs_x = [info['index_x'] for info in info_list]
idxs_y = [info['index_y'] for info in info_list]

bbox_gdf = gpd.GeoDataFrame({'index': idxs, 'index_x': idxs_x, 'index_y': idxs_y}, 
                            crs=UTM33N, geometry=geometry)

df = pd.DataFrame({'index_x':idxs_x, 'index_y':idxs_y})
patches_gdf = gpd.GeoDataFrame(df, crs=UTM33N, geometry=geometry)

In [ ]:
patch_ids = [index for (index, patch) in patches_gdf.iterrows() if harvest_aoi.geometry[0].intersects(patch.geometry)]

In [ ]:
fontdict = {'family': 'monospace', 'weight': 'normal', 'size': 30}
fontdict1 = {'family': 'monospace', 'weight': 'normal', 'size': 20}
# if bboxes have all same size, estimate offset
xl, yl, xu, yu = patches_gdf.geometry[0].bounds
xoff, yoff = (xu-xl)/4, (yu-yl)/4          #<--------- Why 3 and 5?
# figure
fig, ax = plt.subplots(figsize=(25,25))
patches_gdf.plot(ax=ax,facecolor='w',edgecolor='r',alpha=0.5)
sweden_gdf.plot(ax=ax, facecolor='w',edgecolor='b',alpha=0.5)
ax.set_title('Sweden tiled in a 15 x 35 grid');

# add annotiation text
for idx in patches_gdf.index:
    eop_name = '{0}x{1}'.format(patches_gdf.index_x[idx], patches_gdf.index_y[idx])
    centroid, = list(patches_gdf.geometry[idx].centroid.coords)
    ax.text(centroid[0]-xoff, centroid[1]+yoff/2, '{}'.format(idx), fontdict=fontdict)
    ax.text(centroid[0]-xoff, centroid[1]-yoff, eop_name, fontdict=fontdict1)

# Mark bboxes of selected area
# Run below first!!!
bbox_gdf[bbox_gdf.index.isin(patch_ids)].plot(ax=ax, facecolor='g', edgecolor='r', alpha=0.5)
ax.set_xlim([300000, 600000])
ax.set_ylim([6100000, 6300000])
plt.xticks(fontsize=34)
plt.yticks(fontsize=34)

# SentinelHub-credentials

In [ ]:
INSTANCE_ID = ''
CLIENT_ID = ''
CLIENT_SECRET = ''

config = SHConfig()

if CLIENT_ID and CLIENT_SECRET and INSTANCE_ID:
    config.instance_id = INSTANCE_ID
    config.sh_client_id = CLIENT_ID
    config.sh_client_secret = CLIENT_SECRET

if config.sh_client_id == '' or config.sh_client_secret == '' or config.instance_id == '':
    print("Warning! To use Sentinel Hub services, please provide the credentials (client ID and client secret).")

# Main download function
 - By default we download the IW product with orthorectification
 - Max supported resolution is 11x11$m^2$

In [ ]:
def download_data(time_interval, save_path, resolution, processing_params=None):
    create_task = CreateEOPatchTask()
    create_node = EONode(create_task, inputs=[], name="create eo-patch")
    
    if not processing_params:
        # Default params, contains orthorectifaction and radiometric calibration parameters
        processing_params = {
            "backCoeff": "GAMMA0_TERRAIN",
            "orthorectify": True,
            "demInstance": "COPERNICUS",
            "downsampling": "BILINEAR",
            "upsampling": "BILINEAR"
        }

    bands = ["VH", "VV"] # seems to return in alphabetical order
    # bands = ["VV", "VH", "HH", "HV"] NOTE! Not supported
    data_task = SentinelHubInputTask(
        data_collection=DataCollection.SENTINEL1_IW,
        bands_feature=(FeatureType.DATA, "IW"),
        bands=bands,
        config=config,
        time_difference=datetime.timedelta(minutes=120),
        max_threads=4,
        resolution=resolution,
        aux_request_args = {'processing': processing_params}
    )
    data_node = EONode(data_task, inputs=[create_node], name="data eo-patch")

    save_task = SaveTask(save_path, compress_level=1, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)
    save_node = EONode(save_task, inputs=[data_node], name="save eo-patch")

    workflow = EOWorkflow([create_node, data_node, save_node])


    for idx in tqdm(patch_ids):
        bbox = bbox_splitter.bbox_list[idx]
        info = bbox_splitter.info_list[idx]
        patch_name = f'eopatch_{idx}'
        print(patch_name)
        if os.path.exists(os.path.join(save_path, patch_name)):
            print("patch is already downloaded, moving on ...")
            print()
            continue
        try:
            print("executing workflow")
            workflow.execute({
                create_node: {'bbox': bbox},
                data_node: {'time_interval': time_interval},
                save_node: {'eopatch_folder': patch_name}
            })
            print("executing workflow, done!")
                
        except Exception as ex:
            print(f'Failed {patch_name} with {ex}')
        print()

# Customize download settings
 - Time period, save paths etc
 - Stores matrix with shape (time, y, x, polarization), polarization is in order VH, VV

In [ ]:
def download():
    years = [
        #2017,
        #2018,
        #2019,
        2020,
        #2021,
        #2022
    ]

    # April to end of July
    start_mm = "04"
    start_dd = "01"
    end_mm = "07"
    end_dd = "30"

    SENTINEL_1_PATH = SENTINEL_1_11x11_PATH
    for year in years:
        time_interval =  [f"{year}-{start_mm}-{start_dd}", f"{year}-{end_mm}-{end_dd}"]
        save_path = os.path.join(SENTINEL_1_PATH, str(year))
        resolution = 11.0
        print(time_interval)
        print(f"resolution: {resolution} m")
        print(f"starting download: {save_path}")
        download_data(time_interval, save_path, resolution)
        
#download()
